In [1]:
%reset -f

import pandas as pd
import os
import numpy as np
from six.moves import xrange
import math
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [2]:
all_data = pd.read_csv('../data/step1_clear_data.csv')
all_data.head()

,Unnamed: 0,day,exc_num,exc_times,uid,velocity,force,front
0,0,1,1.1,1,4,0.021297,18.561272,0.011647
1,1,1,1.1,1,4,0.021297,18.561272,0.011647
2,2,1,1.1,1,4,0.023768,23.049513,0.011971
3,3,1,1.1,1,4,0.023768,23.049513,0.012333
4,4,1,1.1,1,4,0.031793,32.181287,0.012333
